In [ ]:
### Data
import xarray as xr
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, interplevel, smooth2d

### Plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
### Read in 4-km data

zl_4km_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/z400_11-15_12.npy')
zl_4km_adj = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_full/z400_11-15_12.npy')

pvl_4km_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/pv400_11-15_12.npy')
pvl_4km_adj = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_full/pv400_11-15_12.npy')

pvlay_4km_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/pv300-500_11-15_12.npy')
pvlay_4km_adj = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_full/pv300-500_11-15_12.npy')

lats_4km = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/lats.npy')
lons_4km = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/lons.npy')


In [ ]:
### 30-km data
nx = 400
ny = 250
nz = 44
ti = 16
lev = 400
levs = np.arange(300,510,10)

### Read in the data
ctl = Dataset('/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
adj = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_full/wrfout_d01_2018-11-13_12_00_00')

### Latitude and longitude
lats = np.asarray(getvar(ctl,'lat',timeidx=ti))
lons = np.asarray(getvar(ctl,'lon',timeidx=ti))
lons[lons > 0] -= 360

### Pressure
p_ctl = np.asarray(getvar(ctl,'pressure',timeidx=ti))
p_adj = np.asarray(getvar(adj,'pressure',timeidx=ti))

### Height
z_ctl = np.asarray(getvar(ctl,'z',timeidx=ti))
z_adj = np.asarray(getvar(adj,'z',timeidx=ti))

### PV
pv_ctl = np.asarray(getvar(ctl,'pvo',timeidx=ti))
pv_adj = np.asarray(getvar(adj,'pvo',timeidx=ti))

### Interpolate
zl_ctl = interplevel(z_ctl, p_ctl, lev)
zl_adj = interplevel(z_adj, p_adj, lev)

pvl_ctl = interplevel(pv_ctl, p_ctl, lev)
pvl_adj = interplevel(pv_adj, p_adj, lev)

zlay_ctl = np.mean(interplevel(z_ctl, p_ctl, levs),axis=0)
zlay_adj = np.mean(interplevel(z_adj, p_adj, levs),axis=0)

pvlay_ctl = np.mean(interplevel(pv_ctl, p_ctl, levs),axis=0)
pvlay_adj = np.mean(interplevel(pv_adj, p_adj, levs),axis=0)

In [ ]:
### Plot subdomain
min_lat = 24
max_lat = 45
min_lon = -105
max_lon = -75

### Contour intervals
zlvls = np.arange(0,1500,10)
dzlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*0.75

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-101,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['left','right'],['cbar','cbar']],
                              constrained_layout=True, figsize=(6.9,3.6), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1],'height_ratios':[1,0.05]},
                              per_subplot_kw={'left':{'projection':proj},
                                              'right':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### LEFT
#############

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.right_labels=False
gls.rotate_labels = False

axd['left'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())

cs1 = axd['left'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['left'].contour(lons, lats, smooth2d(zl_adj,1)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['left'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['left'].contourf(lons, lats, pvlay_adj - pvlay_ctl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',zorder=-20,
             transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['left'].set_title('30-km Control vs. 30-km Perturbed')
axd['left'].text(0.0, 1.0, '(a)',transform=axd['left'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### RIGHT
#############

# Add the gridlines
gls = axd['right'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False
axd['right'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())

cs1 = axd['right'].contour(lons_4km, lats_4km, smooth2d(zl_4km_ctl,10)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['right'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['right'].contour(lons_4km, lats_4km, smooth2d(zl_4km_adj,10)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['right'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['right'].contourf(lons_4km, lats_4km, pvlay_4km_adj - pvlay_4km_ctl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',zorder=-20,
             transform=crs.PlateCarree())
axd['right'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['right'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['right'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['right'].set_title('4-km Control vs. 4-km Perturbed')
axd['right'].text(0.0, 1.0, '(b)',transform=axd['right'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig10_wrf_30km_4km.pdf',bbox_inches='tight')
plt.show()
